In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.linear_model import LinearRegression as Lin_Reg
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn import discriminant_analysis
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn import tree
from sklearn import ensemble
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
import StringIO
from sklearn.linear_model import LogisticRegression
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import scipy as sp
import itertools as it
from sklearn import preprocessing
from sklearn import discriminant_analysis
from sklearn.cross_validation import KFold
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn import ensemble
from sklearn import svm
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline

%matplotlib inline

In [2]:
# read in all of the data: regular season compact, regular season detailed, seasons, teams
# tourney compact, tourney detailed, tourney seeds, tourney slots 
regcompact = pd.read_csv("RegularSeasonCompactResults.csv")
regdetailed = pd.read_csv("RegularSeasonDetailedResults.csv")
seasons = pd.read_csv("Seasons.csv")
teams = pd.read_csv("Teams.csv")
tourneycompact = pd.read_csv("TourneyCompactResults.csv")
tourneydetailed = pd.read_csv("TourneyDetailedResults.csv")
tourneyseeds = pd.read_csv("TourneySeeds.csv")
tourneyslots = pd.read_csv("TourneySlots.csv")
kenpom = pd.read_csv("KenPomCSV.csv")



In [3]:
# Create an ID for the winning and losing teams in the regular detailed and tourney detailed 

regdetailed['WteamID'] = regdetailed['Wteam'].map(str) + regdetailed['Season'].map(str)
regdetailed['LteamID'] = regdetailed['Lteam'].map(str) + regdetailed['Season'].map(str)

tourneydetailed['WteamID'] = tourneydetailed['Wteam'].map(str) + tourneydetailed['Season'].map(str)
tourneydetailed['LteamID'] = tourneydetailed['Lteam'].map(str) + tourneydetailed['Season'].map(str)

In [4]:
# just check out the layout

print kenpom.head(n=3)

     TeamName  Year              ID    Tempo  RankTempo  AdjTempo  \
0        Duke  2002        Duke2002  78.6982          6   77.6765   
1  Cincinnati  2002  Cincinnati2002  68.7626        201   68.7952   
2    Maryland  2002    Maryland2002  75.4652         20   74.7618   

   RankAdjTempo       OE  RankOE    AdjOE  RankAdjOE       DE  RankDE  \
0             5  115.544       2  118.671          1  89.8718       4   
1           206  111.852      14  115.077          7  86.0312       1   
2            20  113.028       6  116.170          5  94.0118      35   

     AdjDE  RankAdjDE       EM  RankEM  
0  85.7040          2  32.9668       1  
1  85.3609          1  29.7164       2  
2  89.1643         11  27.0056       3  


In [5]:
# Add the Seed for Each Team

for i in range(0,len(tourneydetailed)):

    tourneydetailed.ix[i,"LKen_Name"] = teams['Ken_Name'][teams['Team_Id']==tourneydetailed.ix[i,'Lteam']].values[0]
    tourneydetailed.ix[i,"WKen_Name"] = teams['Ken_Name'][teams['Team_Id']==tourneydetailed.ix[i,'Wteam']].values[0]

In [6]:
# there were two names that changed over the years

tourneydetailed.ix[32,"LKen_Name"] = "Troy St."
tourneydetailed.ix[647,"LKen_Name"] = "Middle Tennessee"


In [7]:
# merge the kenpom data with the tournament data 

for i in range(0,len(tourneydetailed)):
    tourneydetailed.ix[i,"LKen"] = kenpom['EM'][(kenpom['TeamName']==tourneydetailed.ix[i,'LKen_Name']) & (kenpom['Year']==tourneydetailed.ix[i,'Season'])].values[0]
    tourneydetailed.ix[i,"WKen"] = kenpom['EM'][(kenpom['TeamName']==tourneydetailed.ix[i,'WKen_Name']) & (kenpom['Year']==tourneydetailed.ix[i,'Season'])].values[0]
    
    tourneydetailed.ix[i,"LAdjDE"] = kenpom['AdjDE'][(kenpom['TeamName']==tourneydetailed.ix[i,'LKen_Name']) & (kenpom['Year']==tourneydetailed.ix[i,'Season'])].values[0]
    tourneydetailed.ix[i,"WAdjDE"] = kenpom['AdjDE'][(kenpom['TeamName']==tourneydetailed.ix[i,'WKen_Name']) & (kenpom['Year']==tourneydetailed.ix[i,'Season'])].values[0]
    
    tourneydetailed.ix[i,"LAdjOE"] = kenpom['AdjOE'][(kenpom['TeamName']==tourneydetailed.ix[i,'LKen_Name']) & (kenpom['Year']==tourneydetailed.ix[i,'Season'])].values[0]
    tourneydetailed.ix[i,"WAdjOE"] = kenpom['AdjOE'][(kenpom['TeamName']==tourneydetailed.ix[i,'WKen_Name']) & (kenpom['Year']==tourneydetailed.ix[i,'Season'])].values[0]
    
    tourneydetailed.ix[i,"LTempo"] = kenpom['AdjTempo'][(kenpom['TeamName']==tourneydetailed.ix[i,'LKen_Name']) & (kenpom['Year']==tourneydetailed.ix[i,'Season'])].values[0]
    tourneydetailed.ix[i,"WTempo"] = kenpom['AdjTempo'][(kenpom['TeamName']==tourneydetailed.ix[i,'WKen_Name']) & (kenpom['Year']==tourneydetailed.ix[i,'Season'])].values[0]
    
    tourneydetailed.ix[i,"LDERank"] = kenpom['RankAdjDE'][(kenpom['TeamName']==tourneydetailed.ix[i,'LKen_Name']) & (kenpom['Year']==tourneydetailed.ix[i,'Season'])].values[0]
    tourneydetailed.ix[i,"WDERank"] = kenpom['RankAdjDE'][(kenpom['TeamName']==tourneydetailed.ix[i,'WKen_Name']) & (kenpom['Year']==tourneydetailed.ix[i,'Season'])].values[0]
    
    tourneydetailed.ix[i,"LOERank"] = kenpom['RankAdjOE'][(kenpom['TeamName']==tourneydetailed.ix[i,'LKen_Name']) & (kenpom['Year']==tourneydetailed.ix[i,'Season'])].values[0]
    tourneydetailed.ix[i,"WOERank"] = kenpom['RankAdjOE'][(kenpom['TeamName']==tourneydetailed.ix[i,'WKen_Name']) & (kenpom['Year']==tourneydetailed.ix[i,'Season'])].values[0]
    
    

In [8]:
# Create a Team_ID for the tourneyseed

tourneyseeds['Team_Id'] = tourneyseeds['Team'].map(str) + tourneyseeds['Season'].map(str)

In [9]:
# Add the Seed for Each Team

for i in range(0,len(tourneydetailed)):

    tourneydetailed.ix[i,"LSeed"] = tourneyseeds['Seed'][tourneyseeds['Team_Id']==tourneydetailed.ix[i,'LteamID']].values[0]
    tourneydetailed.ix[i,"WSeed"] = tourneyseeds['Seed'][tourneyseeds['Team_Id']==tourneydetailed.ix[i,'WteamID']].values[0]
    
    

In [10]:
# Get the No. seed from the total seed 

for i in range(0,len(tourneydetailed)):
    if (len(tourneydetailed.ix[i,"WSeed"])==3):
        tourneydetailed.ix[i,"WSeedNum"] = int(tourneydetailed.ix[i,"WSeed"][-2:])
    else: 
        tourneydetailed.ix[i,"WSeedNum"] = int(tourneydetailed.ix[i,"WSeed"][1:3])
    if (len(tourneydetailed.ix[i,"LSeed"])==3):
        tourneydetailed.ix[i,"LSeedNum"] = int(tourneydetailed.ix[i,"LSeed"][-2:])
    else: 
        tourneydetailed.ix[i,"LSeedNum"] = int(tourneydetailed.ix[i,"LSeed"][1:3])

In [11]:
# some of these are the "play-in" games (like 16 vs. 16). just making a note of those

for i in range(0,len(tourneydetailed)):
    if (tourneydetailed.ix[i,"WSeed"][0:3]==tourneydetailed.ix[i,"LSeed"][0:3]):
        tourneydetailed.ix[i,"Play-In"] = 1
    else:
        tourneydetailed.ix[i,"Play-In"] = 0

In [12]:
# calculate how many field goals they assist on 

regdetailed['Wastratio'] = regdetailed['Wast'] / regdetailed['Wfgm']
regdetailed['Lastratio'] = regdetailed['Last'] / regdetailed['Lfgm']

In [13]:
# calculate dr rate

regdetailed['Wdrratio'] = 1 / ((regdetailed['Lfga'] - regdetailed['Lfgm']) / regdetailed['Wdr'])
regdetailed['Ldrratio'] = 1 / ((regdetailed['Wfga'] - regdetailed['Wfgm']) / regdetailed['Ldr'])

In [14]:
# calculate or rate 

regdetailed['Worratio'] = 1 / ((regdetailed['Wfga'] - regdetailed['Wfgm']) / regdetailed['Wor'])
regdetailed['Lorratio'] = 1 / ((regdetailed['Lfga'] - regdetailed['Lfgm']) / regdetailed['Lor'])

In [15]:
# calculate point diff

regdetailed['ptdiff'] = ((regdetailed['Wscore'] - regdetailed['Lscore']))

In [16]:
# calc 2pt fg %, 3pt fg %

regdetailed['W2fgper'] = (regdetailed['Wfgm'] - regdetailed['Wfgm3']) / (regdetailed['Wfga'] - regdetailed['Wfga3'])
regdetailed['W3fgper'] = (regdetailed['Wfgm3'] / regdetailed['Wfga3'])

regdetailed['L2fgper'] = (regdetailed['Lfgm'] - regdetailed['Lfgm3']) / (regdetailed['Lfga'] - regdetailed['Lfga3'])
regdetailed['L3fgper'] = (regdetailed['Lfgm3'] / regdetailed['Lfga3'])

In [17]:
for i in range(0,len(tourneydetailed)):
    if (tourneydetailed.ix[i,"Play-In"]==1):
        tourneydetailed.ix[i,'round'] = 0
    elif ((tourneydetailed.ix[i,"Daynum"]==136) or (tourneydetailed.ix[i,"Daynum"]==137)):
        tourneydetailed.ix[i,'round'] = 1
    elif ((tourneydetailed.ix[i,"Daynum"]==138) or (tourneydetailed.ix[i,"Daynum"]==139)):
        tourneydetailed.ix[i,'round'] = 2
    elif ((tourneydetailed.ix[i,"Daynum"]==143) or (tourneydetailed.ix[i,"Daynum"]==144)):
        tourneydetailed.ix[i,'round'] = 3
    elif ((tourneydetailed.ix[i,"Daynum"]==145) or (tourneydetailed.ix[i,"Daynum"]==146)):
        tourneydetailed.ix[i,'round'] = 4
    elif (tourneydetailed.ix[i,"Daynum"]==152):
        tourneydetailed.ix[i,'round'] = 5
    elif (tourneydetailed.ix[i,"Daynum"]==154):
        tourneydetailed.ix[i,'round'] = 6

    

In [18]:
# dont worry about this for now

def simple_ranking_system(year):
    ranking_array = pd.DataFrame(np.zeros((len(regdetailed[regdetailed['Season']==year]),365)))
    for i in range(0,len(regdetailed[regdetailed['Season']==year])):
        ranking_array.iloc[i,regdetailed.ix[i,'Wteam']-1101] = 1
        ranking_array.iloc[i,regdetailed.ix[i,'Lteam']-1101] = -1
        # ranking_array.iloc[i,364] = regdetailed.ix[i,'ptdiff']
        ranking_array.iloc[i,364] = 1
    regression = Lin_Reg()
    regression.fit(ranking_array.iloc[:,range(1,364)],ranking_array.iloc[:,364])
    return np.append(regression.intercept_, regression.coef_)
        
    

In [19]:
# calculate the wins and losses for each team

wins_each_year = pd.DataFrame(np.zeros((14,364)))
losses_each_year = pd.DataFrame(np.zeros((14,364)))

for i in range(2003,2017):
    for j in range(1101,1465):
        wins_each_year.iloc[i-2003,j-1101] = len(regdetailed[(regdetailed['Wteam']==j) & (regdetailed['Season']==i)])
        losses_each_year.iloc[i-2003,j-1101] = len(regdetailed[(regdetailed['Lteam']==j) & (regdetailed['Season']==i)])



In [20]:
# use this as a key to bring statistics into a prediction matrix

winstats = ['Wscore', 'W2fgper', 'W3fgper','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf','Wastratio','Wdrratio','Worratio','Lscore', 'L2fgper', 'L3fgper','Lfta','Lor','Ldr','Last','Lto','Lstl','Lblk','Lpf','Lastratio','Ldrratio','Lorratio']
losestats = ['Lscore', 'L2fgper', 'L3fgper','Lfta','Lor','Ldr','Last','Lto','Lstl','Lblk','Lpf','Lastratio','Ldrratio','Lorratio','Wscore', 'W2fgper', 'W3fgper','Wfta','Wor','Wdr','Wast','Wto','Wstl','Wblk','Wpf','Wastratio','Wdrratio','Worratio']
col_names = ['scorefor','2fgperfor', '3fgperfor','ftafor','orfor','drfor','astfor','tofor','stlfor','blkfor','pffor','astratiofor','drratiofor','orratiofor','scoreag','2fgperag', '3fgperag','ftaag','orag','drag','astag','toag','stlag','blkag','pfag','astratioag','drratioag','orratioag']

d = {'winstats': winstats, 'losestats': losestats,'col_names': col_names}
key = pd.DataFrame(data=d)



In [21]:
# accumulate reg season stats of winner, loser

new_tourney_stats_winner = pd.DataFrame(np.zeros((len(tourneydetailed),28)))
new_tourney_stats_loser = pd.DataFrame(np.zeros((len(tourneydetailed),28)))

for i in range(0,len(new_tourney_stats_winner)):
    for j in range(0,28):
        new_tourney_stats_winner.iloc[i,j] = np.mean(np.append(regdetailed[key.iloc[j,2]]
                                            [(regdetailed['Wteam']==tourneydetailed.ix[i,'Wteam']) & 
                                           (regdetailed['Season']==tourneydetailed.ix[i,'Season'])],
                                            regdetailed[key.iloc[j,1]]
                                            [(regdetailed['Lteam']==tourneydetailed.ix[i,'Wteam']) & 
                                           (regdetailed['Season']==tourneydetailed.ix[i,'Season'])]))

for i in range(0,len(new_tourney_stats_loser)):
    for j in range(0,28):
        new_tourney_stats_loser.iloc[i,j] = np.mean(np.append(regdetailed[key.iloc[j,2]]
                                            [(regdetailed['Wteam']==tourneydetailed.ix[i,'Lteam']) & 
                                           (regdetailed['Season']==tourneydetailed.ix[i,'Season'])],
                                            regdetailed[key.iloc[j,1]]
                                            [(regdetailed['Lteam']==tourneydetailed.ix[i,'Lteam']) & 
                                           (regdetailed['Season']==tourneydetailed.ix[i,'Season'])]))


In [27]:
# we want to determine failures and successes. Therefore, we will call a game a success if the higher ranked team wins 
# so, we have to reorganize data from winning seed and losing seed to high seed and low seed

prediction_matrix = pd.DataFrame(np.zeros((len(tourneydetailed),71)))

for i in range(0,len(tourneydetailed)):
    if(tourneydetailed.ix[i,"WSeedNum"] > tourneydetailed.ix[i,"LSeedNum"]):
        x = pd.concat([new_tourney_stats_loser.iloc[i,:], pd.Series(tourneydetailed.ix[i,'LKen'])
                       ,pd.Series(tourneydetailed.ix[i,'LAdjOE']),
                       pd.Series(tourneydetailed.ix[i,'LAdjDE']),pd.Series(tourneydetailed.ix[i,'LTempo']),
                       pd.Series(tourneydetailed.ix[i,'LDERank']),pd.Series(tourneydetailed.ix[i,'LOERank']),
                       new_tourney_stats_winner.iloc[i,:],
                       pd.Series(tourneydetailed.ix[i,'WKen']),pd.Series(tourneydetailed.ix[i,'WAdjOE']),
                       pd.Series(tourneydetailed.ix[i,'WAdjDE']),pd.Series(tourneydetailed.ix[i,'WTempo']),
                       pd.Series(tourneydetailed.ix[i,'WDERank']),pd.Series(tourneydetailed.ix[i,'WOERank']),
                       pd.Series(tourneydetailed.ix[i,'LSeedNum']),pd.Series(tourneydetailed.ix[i,'WSeedNum']),pd.Series(tourneydetailed.ix[i,'round'])], axis=0)
        prediction_matrix.iloc[i,:] = np.array(x)
        # prediction_matrix.iloc[i,range(0,28)] = new_tourney_stats_loser.iloc[i,:]
        # prediction_matrix.iloc[i,range(28,56)] = new_tourney_stats_winner.iloc[i,:]
        # prediction_matrix.iloc[i,56] = tourneydetailed.ix[i,'LSeedNum']
        # prediction_matrix.iloc[i,57] = tourneydetailed.ix[i,'WSeedNum']
    else:
        x = pd.concat([new_tourney_stats_winner.iloc[i,:], 
               pd.Series(tourneydetailed.ix[i,'WKen']),pd.Series(tourneydetailed.ix[i,'WAdjOE']),
               pd.Series(tourneydetailed.ix[i,'WAdjDE']),pd.Series(tourneydetailed.ix[i,'WTempo']),
                pd.Series(tourneydetailed.ix[i,'WDERank']),pd.Series(tourneydetailed.ix[i,'WOERank']),
               new_tourney_stats_loser.iloc[i,:],
               pd.Series(tourneydetailed.ix[i,'LKen']),pd.Series(tourneydetailed.ix[i,'LAdjOE']),
               pd.Series(tourneydetailed.ix[i,'LAdjDE']),pd.Series(tourneydetailed.ix[i,'LTempo']),
                pd.Series(tourneydetailed.ix[i,'LDERank']),pd.Series(tourneydetailed.ix[i,'LOERank']),
               pd.Series(tourneydetailed.ix[i,'WSeedNum']),pd.Series(tourneydetailed.ix[i,'LSeedNum']),pd.Series(tourneydetailed.ix[i,'round'])], axis=0)
        prediction_matrix.iloc[i,:] = np.array(x)
        # prediction_matrix.iloc[i,range(0,28)] = new_tourney_stats_winner.iloc[i,:]
        # prediction_matrix.iloc[i,range(28,56)] = new_tourney_stats_loser.iloc[i,:]
        # prediction_matrix.iloc[i,56] = tourneydetailed.ix[i,'WSeedNum']
        # prediction_matrix.iloc[i,57] = tourneydetailed.ix[i,'LSeedNum']
    

prediction_matrix.cols = ['team1scorefor','team12fgperfor', 'team13fgperfor','team1ftafor','team1orfor','team1drfor','team1astfor','team1tofor','team1stlfor','team1blkfor','team1pffor','team1astratiofor','team1drratiofor','team1orratiofor','team1scoreag','team12fgperag', 'team13fgperag','team1ftaag','team1orag','team1drag','team1astag','team1toag','team1stlag','team1blkag','team1pfag','team1astratioag','team1drratioag','team1orratioag','team1ken', 'team1kenoe','team1kende', 'team1kentempo','team1drank','team1orank','team2scorefor','team22fgperfor', 'team23fgperfor','team2ftafor','team2orfor','team2drfor','team2astfor','team2tofor','team2stlfor','team2blkfor','team2pffor','team2astratiofor','team2drratiofor','team2orratiofor','team2scoreag','team22fgperag', 'team23fgperag','team2ftaag','team2orag','team2drag','team2astag','team2toag','team2stlag','team2blkag','team2pfag','team2astratioag','team2drratioag','team2orratioag','team2ken', 'team2kenoe','team2kende', 'team2kentempo','team2drank','team2orank','team1seed','team2seed','round']

#prediction_matrix = pd.DataFrame(prediction_matrix, columns=new_cols)    
    # this is for classification
    
prediction_answers =pd.DataFrame(np.zeros((len(tourneydetailed),1)))  
for i in range(0,len(tourneydetailed)):
    if(tourneydetailed.ix[i,"WSeedNum"]<tourneydetailed.ix[i,"LSeedNum"]):
        prediction_answers.iloc[i,0] = 1
        
# this is for point differential
        
prediction_answers2 =pd.DataFrame(np.zeros((len(tourneydetailed),1)))  
for i in range(0,len(tourneydetailed)):
    if(tourneydetailed.ix[i,"WSeedNum"]<tourneydetailed.ix[i,"LSeedNum"]):
        prediction_answers2.iloc[i,0] = tourneydetailed.ix[i,"Wscore"] - tourneydetailed.ix[i,"Lscore"]
    else:
        prediction_answers2.iloc[i,0] = tourneydetailed.ix[i,"Lscore"] - tourneydetailed.ix[i,"Wscore"]

In [28]:
# run a train / test to split data

# x_train, x_test, y_train_val, y_test_val = train_test_split(prediction_matrix[tourneydetailed["Play-In"]==0], np.array(prediction_answers[tourneydetailed["Play-In"]==0]), test_size=0.85)
x_train = prediction_matrix[(tourneydetailed["Play-In"]==0) & (tourneydetailed["Season"] < 2014)]
x_test = prediction_matrix[(tourneydetailed["Play-In"]==0) & (tourneydetailed["Season"] > 2013)]
y_train_val = prediction_answers[(tourneydetailed["Play-In"]==0) & (tourneydetailed["Season"] < 2014)]

y_test_val = prediction_answers[(tourneydetailed["Play-In"]==0) & (tourneydetailed["Season"] > 2013)]

In [29]:
from sklearn import tree

dt = tree.DecisionTreeClassifier()
dt.fit(x_train,y_train_val)
print "The DT classifier produces a score of ", round(dt.score(x_test, y_test_val),2)

rf = ensemble.RandomForestClassifier()
rf.fit(x_train,y_train_val)
print "The RF classifier produces a score of ", round(rf.score(x_test, y_test_val),2)

# lda fit and score

lda = discriminant_analysis.LinearDiscriminantAnalysis()
lda.fit(x_train,y_train_val)
print "The LDA classifier produces a score of ", round(lda.score(x_test, y_test_val),2)

qda = discriminant_analysis.QuadraticDiscriminantAnalysis()
qda.fit(x_train,y_train_val)
print "The QDA classifier produces a score of ", round(qda.score(x_test, y_test_val),2)

# Fit SVM model 
svm_model = svm.SVC()
svm_model.fit(x_train,y_train_val)
print "The SVM classifier produces a score of ", round(svm_model.score(x_test, y_test_val),2)

logistic_balanced = LogisticRegression()
logistic_balanced.fit(x_train,y_train_val)
print "The LogReg classifier produces a score of ", round(logistic_balanced.score(x_test, y_test_val),2)

neighbors = KNN(4)
neighbors.fit(x_train,y_train_val)
print "The KNN classifier produces a score of ", round(neighbors.score(x_test, y_test_val),2)

The DT classifier produces a score of  0.55
The RF classifier produces a score of  0.65
The LDA classifier produces a score of  0.69
The QDA classifier produces a score of  0.7


/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/ana

The SVM classifier produces a score of  0.71
The LogReg classifier produces a score of  0.72
The KNN classifier produces a score of  0.7


/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:31: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [30]:
from sklearn.preprocessing import PolynomialFeatures

x_train_int = PolynomialFeatures(2, interaction_only=True, include_bias=False).fit_transform(x_train)
x_test_int = PolynomialFeatures(2, interaction_only=True, include_bias=False).fit_transform(x_test)


dt = tree.DecisionTreeClassifier(max_depth = 10)
dt.fit(x_train_int,y_train_val)
print "The DT classifier produces a score of ", round(dt.score(x_test_int, y_test_val),2)

rf = ensemble.RandomForestClassifier()
rf.fit(x_train_int,y_train_val)
print "The RF classifier produces a score of ", round(rf.score(x_test_int, y_test_val),2)

# lda fit and score

lda = discriminant_analysis.LinearDiscriminantAnalysis()
lda.fit(x_train_int,y_train_val)
print "The LDA classifier produces a score of ", round(lda.score(x_test_int, y_test_val),2)

qda = discriminant_analysis.QuadraticDiscriminantAnalysis()
qda.fit(x_train_int,y_train_val)
print "The QDA classifier produces a score of ", round(qda.score(x_test_int, y_test_val),2)

# Fit SVM model 
svm_model = svm.SVC()
svm_model.fit(x_train_int,y_train_val)
print "The SVM classifier produces a score of ", round(svm_model.score(x_test_int, y_test_val),2)

logistic_balanced = LogisticRegression()
logistic_balanced.fit(x_train_int,y_train_val)
print "The LogReg classifier produces a score of ", round(logistic_balanced.score(x_test_int, y_test_val),2)

neighbors = KNN(4)
neighbors.fit(x_train_int,y_train_val)
print "The KNN classifier produces a score of ", round(neighbors.score(x_test_int, y_test_val),2)

The DT classifier produces a score of  0.56


/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


The RF classifier produces a score of  0.67


/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The LDA classifier produces a score of  0.6


...

The SVM classifier produces a score of  0.71


/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The LogReg classifier produces a score of  0.61
The KNN classifier produces a score of 

/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


 0.73


In [31]:
from sklearn.linear_model import Lasso as Lasso_Reg

reg = Lasso_Reg()
reg.fit(x_train_int, y_train_val)
coefficients = reg.coef_

/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


In [32]:
x_train_int = pd.DataFrame(x_train_int)
x_test_int = pd.DataFrame(x_test_int)

x_train_trim = x_train_int.iloc[:,non_zeros]
x_test_trim = x_test_int.iloc[:,non_zeros]

# print the column names 

x_train_trim.columns.values

NameError: name 'non_zeros' is not defined

In [33]:
from sklearn.preprocessing import PolynomialFeatures

x_train_int = PolynomialFeatures(2, interaction_only=True, include_bias=False).fit_transform(x_train)
x_test_int = PolynomialFeatures(2, interaction_only=True, include_bias=False).fit_transform(x_test)


dt = tree.DecisionTreeClassifier(max_depth = 10)
dt.fit(x_train_trim,y_train_val)
print "The DT classifier produces a score of ", round(dt.score(x_test_trim, y_test_val),2)

rf = ensemble.RandomForestClassifier()
rf.fit(x_train_trim,y_train_val)
print "The RF classifier produces a score of ", round(rf.score(x_test_trim, y_test_val),2)

# lda fit and score

lda = discriminant_analysis.LinearDiscriminantAnalysis()
lda.fit(x_train_trim,y_train_val)
print "The LDA classifier produces a score of ", round(lda.score(x_test_trim, y_test_val),2)

qda = discriminant_analysis.QuadraticDiscriminantAnalysis()
qda.fit(x_train_trim,y_train_val)
print "The QDA classifier produces a score of ", round(qda.score(x_test_trim, y_test_val),2)

# Fit SVM model 
svm_model = svm.SVC()
svm_model.fit(x_train_trim,y_train_val)
print "The SVM classifier produces a score of ", round(svm_model.score(x_test_trim, y_test_val),2)

logistic_balanced = LogisticRegression()
logistic_balanced.fit(x_train_trim,y_train_val)
print "The LogReg classifier produces a score of ", round(logistic_balanced.score(x_test_trim, y_test_val),2)

neighbors = KNN(4)
neighbors.fit(x_train_trim,y_train_val)
print "The KNN classifier produces a score of ", round(neighbors.score(x_test_trim, y_test_val),2)

NameError: name 'x_train_trim' is not defined

In [34]:
for i in range(1,10):
    pca = PCA(n_components = i)
    pca.fit(x_train_int)
    x_train_reduced = pca.transform(x_train_int)
    x_test_reduced = pca.transform(x_test_int)
    
    rf = ensemble.RandomForestClassifier()
    rf.fit(x_train_int,y_train_val)
    print "The RF classifier produces a score of ", round(rf.score(x_test_int, y_test_val),2)

    dt = tree.DecisionTreeClassifier()
    dt.fit(x_train_reduced, y_train_val)
    print "The DT classifier produces a score of ", round(dt.score(x_test_reduced, y_test_val),2)

    logistic_balanced = LogisticRegression(C = .01, class_weight="balanced")
    logistic_balanced.fit(x_train_reduced, y_train_val)
    print "The LogReg classifier produces a score of ", round(logistic_balanced.score(x_test_reduced, y_test_val),2)

    lda = discriminant_analysis.LinearDiscriminantAnalysis()
    lda.fit(x_train_reduced, y_train_val)
    print "The LDA classifier produces a score of ", round(lda.score(x_test_reduced, y_test_val),2)

    qda = discriminant_analysis.QuadraticDiscriminantAnalysis()
    qda.fit(x_train_reduced, y_train_val)
    print "The QDA classifier produces a score of ", round(qda.score(x_test_reduced, y_test_val),2)

    # Fit SVM model 
    svm_model = svm.SVC()
    svm_model.fit(x_train_reduced, y_train_val)
    print "The SVM classifier produces a score of ", round(svm_model.score(x_test_reduced, y_test_val),2)

    neighbors = KNN(4)
    neighbors.fit(x_train_reduced, y_train_val)
    print "The KNN classifier produces a score of ", round(neighbors.score(x_test_reduced, y_test_val),2)




/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The RF classifier produces a score of  0.72
The DT classifier produces a score of  0.63
The LogReg classifier produces a score of  0.6
The LDA classifier produces a score of  0.71
The QDA classifier produces a score of  0.71
The SVM classifier produces a score of  0.71
The KNN classifier produces a score of  0.59


/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:33: DataConversionWarning: A column-vector y was passed when a 1d array was expected. P

The RF classifier produces a score of  0.66
The DT classifier produces a score of  0.59
The LogReg classifier produces a score of  0.57
The LDA classifier produces a score of  0.71
The QDA classifier produces a score of  0.64
The SVM classifier produces a score of  0.71
The KNN classifier produces a score of  0.64


/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expec

...

/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expec

The RF classifier produces a score of  0.68
The DT classifier produces a score of  0.66
The LogReg classifier produces a score of  0.56
The LDA classifier produces a score of  0.68
The QDA classifier produces a score of  0.6
The SVM classifier produces a score of  0.71
The KNN classifier produces a score of  0.73


/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expec

The RF classifier produces a score of  0.63
The DT classifier produces a score of  0.71
The LogReg classifier produces a score of  0.56
The LDA classifier produces a score of  0.68
The QDA classifier produces a score of  0.6
The SVM classifier produces a score of  0.71
The KNN classifier produces a score of  0.72


/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kendrickvinar/anaconda/lib/python2.7/site-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expec